In [2]:
import cv2
import numpy as np
from skimage.filters import threshold_local
from skimage import measure

# blob
In Image processing, blob detection refers to modules that are aimed at detecting points and/or regions in the image that differ in properties like brightness or color compared to the surrounding.

BLOB stands for Binary Large OBject and refers to a group of connected pixels in a binary image. The term “Large” indicates that only objects of a certain size are of interest and that “small” binary objects are usually noise.

# Connected-component labeling
Connected-component labeling (also known as connected-component analysis, blob extraction, or region labeling) is an algorithmic application of graph theory that is used to determine the connectivity of “blob”-like regions in a binary image.

We often use connected-component analysis in the same situations that contours are used; however, connected-component labeling can often give us a more granular filtering of the blobs in a binary image. When using contour analysis, we are often restricted by the hierarchy of the outlines (i.e. one contour contained within another), but with connected-component analysis we can more easily segment and analyze these structures.

Once we have extracted the blob using connected-component labeling, we can still apply contour properties to quantify the region. A great example usage of connected-component analysis is to compute the connected-components of a binary (i.e. thresholded) license plate image and filter the blobs based on their properties, such as width, height, area, solidity, etc.

# The classical approach
The classical connected-component analysis was introduced by Rosenfeld and Pfaltz in their 1966 article, Sequential Operations in Digital Picture Processing. Their clever use of graph theory to analyze connected-components in an image is very efficient and is still heavily used today.

It’s important to note that we only apply connected-component analysis to binary or thresholded images. If presented with an RGB or grayscale image, we first need to threshold it based on some criterion in a manner that can segment the background from the foreground, leaving us with “blobs” in the image that we can examine. Once we have obtained the binary version of the image, we can proceed to analyze the components.

The actual algorithm consists of two passes. In the first pass, the algorithm loops over each individual pixel. For each center pixel p, the west and north pixels are checked:

This type of check is called 4-connectivity (left). Based on the west and north pixel labels, a label is assigned to the current center pixel p (we’ll discuss this step in more detail in the first pass section of this lesson).

You might be wondering why only two pixels are being checked if we want to check the pixels surrounding p for 4-connectivity. The reason is because we are looping over each pixel individually and always checking the west and north pixels. By repeating this process over the entire image, one row at a time, each pixel will actually be checked for 4-connectivity.

8-connectivity can also be performed by checking the west, north-west, north, and north-east pixels (right).

Then, in the second pass, the connected-component analysis algorithm loops over the labels generated from the first pass and merges any regions together that share connected labels.

The first pass
In the first pass of our connected-component analysis algorithm, every pixel is checked. For the sake of this example, we’ll use 4-connectivity (but we could just as easily use 8-connectivity) and check the west and north pixels of the central pixel p:

# Step 1
The first step is to check if we care about the central pixel p or not:

If the central pixel is a background pixel (normally a value of 0, indicating black), we ignore it and move to the next pixel.
If it is a foreground pixel, or if we have moved to a pixel that is in the foreground, we proceed to Steps 2 and 3.
# Steps 2 and 3
If we have reached this step, then we must be examining a foreground pixel, so we grab the north and west pixels, denoted as N and W, respectively:

Now that we have N and W, there are two possible situations:

Both N and W are background pixels, so there are no labels associated with these pixels. In this case, create a new label (normally by incrementing a unique label counter) and store the label value in the current pixel. Then move on to Steps 4 and 5.
N and/or W are not background pixels. If this is the case, we can already proceed to Steps 4 and 5 since at least one pixel already has a label associated with it.
# Steps 4 and 5
This is an easy step. All we need to do is set the center pixel p by taking the minimum of the label value:

p = min(N,W)
# Step 6
Suppose that, in the following figure, the north pixel has label X and the west pixel has label Y:

Even though these pixels have two separate labels, we know they are actually connected and part of the same blob. To indicate that the X and Y labels are part of the same component, we can leverage the union-find data structure to indicate that X is a child of Y. We’ll insert a node in our union-find structure to indicate that X is a child of Y and that the pixels are actually connected even though they have different label values.

The second pass of our connected-components algorithm will leverage the union-find structure to connect any blobs that have different labels but are actually part of the same blob.

# Step 7
Continue to the next pixel and go repeat the process beginning with Step 1.

# The second pass
The second pass of the connected-components labeling algorithm is much simpler than the first one. We start off by looping over the image once again, one pixel at a time.

For each pixel, we check if the label of the current pixel is a root (i.e. top of the tree) in the union-find data structure. If so, then we can proceed on to the next step — the label of the current pixel already has the smallest possible value based on how it is connected to its neighbors.

Otherwise, we follow the tree until we reach a root in the structure. Once we have reached a root, we assign the value at the root to the current pixel:

In [3]:
image=cv2.imread("rail.jpg")
HSV=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
V=cv2.split(HSV)[2]
thresh=cv2.adaptiveThreshold(V,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV, 17, 3)
cv2.imshow("Image",image)
cv2.imshow("THRESH",thresh)
cv2.waitKey(0)

-1

In [3]:
# perform connected components analysis on the thresholded images and initialize the
# mask to hold only the "large" components we are interested in
labels = measure.label(thresh, neighbors=8, background=0)
mask = np.zeros(thresh.shape, dtype="uint8")
print("[INFO] found {} blobs".format(len(np.unique(labels))))

[INFO] found 5692 blobs


<ipython-input-3-479f8b91b51b>:3: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=8, use connectivity=2
  labels = measure.label(thresh, neighbors=8, background=0)


In [ ]:
# loop over the unique components
for (i, label) in enumerate(np.unique(labels)):
	# if this is the background label, ignore it
	if label == 0:
		print("[INFO] label: 0 (background)")
		continue
	# otherwise, construct the label mask to display only connected components for
	# the current label
	print("[INFO] label: {} (foreground)".format(i))
	labelMask = np.zeros(thresh.shape, dtype="uint8")
	labelMask[labels == label] = 255
	numPixels = cv2.countNonZero(labelMask)
	# if the number of pixels in the component is sufficiently large, add it to our
	# mask of "large" blobs
	if numPixels > 3000 and numPixels < 4500:
		mask = cv2.add(mask, labelMask)
	# show the label mask
	cv2.imshow("Label", labelMask)
	cv2.waitKey(0)
# show the large components in the image
cv2.imshow("Large Blobs", mask)
cv2.waitKey(0)

[INFO] label: 0 (background)
[INFO] label: 1 (foreground)
[INFO] label: 2 (foreground)
[INFO] label: 3 (foreground)
[INFO] label: 4 (foreground)
[INFO] label: 5 (foreground)
[INFO] label: 6 (foreground)
[INFO] label: 7 (foreground)
[INFO] label: 8 (foreground)
[INFO] label: 9 (foreground)
[INFO] label: 10 (foreground)
[INFO] label: 11 (foreground)
[INFO] label: 12 (foreground)
[INFO] label: 13 (foreground)
[INFO] label: 14 (foreground)
[INFO] label: 15 (foreground)
[INFO] label: 16 (foreground)
[INFO] label: 17 (foreground)
[INFO] label: 18 (foreground)
[INFO] label: 19 (foreground)
[INFO] label: 20 (foreground)
[INFO] label: 21 (foreground)
[INFO] label: 22 (foreground)
[INFO] label: 23 (foreground)
[INFO] label: 24 (foreground)
[INFO] label: 25 (foreground)
[INFO] label: 26 (foreground)
[INFO] label: 27 (foreground)
[INFO] label: 28 (foreground)
[INFO] label: 29 (foreground)
[INFO] label: 30 (foreground)
